<a href="https://colab.research.google.com/github/wly-2020/kaggle_June/blob/main/kaggle_june.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import tensorflow as tf
import random
import os

os.environ['TF_DETERMINISTIC_OPS'] = '1'

SEED=42

random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

train = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv')
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv')

In [ ]:
from sklearn.preprocessing import MinMaxScaler

label_map = {
    'Class_1' : 0,
    'Class_2' : 1,
    'Class_3' : 2,
    'Class_4' : 3,
    'Class_5' : 4,
    'Class_6' : 5,
    'Class_7' : 6,
    'Class_8' : 7,
    'Class_9' : 8,
}
train['target'] = train['target'].map(label_map)

In [ ]:
features = ['feature_{}'.format(x) for x in range(75)]
qt = train[features].quantile(np.arange(0,1,0.002))
def clip(df):
    df = df.copy()
    for feature in features:
        df[feature] = df[feature].clip(lower=0, upper=qt.loc[0.998][feature])
    return df

In [ ]:
values=[]
labels=[0,1,2,3,4,5,6,7,8,]
for feature in features:
    grouped = clip(train).groupby(feature)
    for value, group in grouped:
        value=[feature, value]
        for label in labels:
            p =  (group['target'] == label).mean()
            p = np.clip(p, 1e-06, 1 - 1e-06)
            value.append(np.log(p+0.5))
            value.append(np.log(p/(1-p)))
        values.append(value)
df_proba = pd.DataFrame(values,
                        columns=['feature', 'value',
                                 'Class_1_proba1',
                                 'Class_1_proba2',
                                 'Class_2_proba1',
                                 'Class_2_proba2',
                                 'Class_3_proba1',
                                 'Class_3_proba2',
                                 'Class_4_proba1',
                                 'Class_4_proba2',
                                 'Class_5_proba1',
                                 'Class_5_proba2',
                                 'Class_6_proba1',
                                 'Class_6_proba2',
                                 'Class_7_proba1',
                                 'Class_7_proba2',
                                 'Class_8_proba1',
                                 'Class_8_proba2',
                                 'Class_9_proba1',
                                 'Class_9_proba2',
                                ])
proba_dict_1={}
proba_dict_2={}

for i in range(len(df_proba)):
    feature = df_proba.iloc[i]['feature']
    value = df_proba.iloc[i]['value']
    proba_dict_1[feature, value] = df_proba.iloc[i][['Class_1_proba1','Class_2_proba1','Class_3_proba1','Class_4_proba1','Class_5_proba1','Class_6_proba1','Class_7_proba1','Class_8_proba1','Class_9_proba1',]].values.astype(float)
    proba_dict_2[feature, value] = df_proba.iloc[i][['Class_1_proba2','Class_2_proba2','Class_3_proba2','Class_4_proba2','Class_5_proba2','Class_6_proba2','Class_7_proba2','Class_8_proba2','Class_9_proba2',]].values.astype(float)
    

In [ ]:
from sklearn.base import TransformerMixin

def reshape(df):
    values=[]
    for value in df.values:
        values.append([_ for _ in value])
    return np.array(values)

class MyTransformer1(TransformerMixin):
    def fit_transform(self, X, y=None,**fit_params):
        return self.transform(X)
    def transform(self, X):
        newX = pd.DataFrame()
        for feature in features:
            newX[feature] = X[feature].clip(lower=qt.loc[0.002][feature], upper=qt.loc[0.998][feature])
            newX[feature] = 1 / (newX[feature] - newX[feature].min() + 1)
        return newX

class MyTransformer2(TransformerMixin):
    def fit_transform(self, X, y=None,**fit_params):
        return self.transform(X)
    def transform(self, X):
        newX = pd.DataFrame()
        X = clip(X)
        for feature in features:
            newX[feature] = X[feature].apply(lambda x:proba_dict_1[feature, x])
        return reshape(newX).reshape((-1,75*9))

class MyTransformer3(MyTransformer2):
    def fit_transform(self, X, y=None,**fit_params):
        return self.transform(X)
    def transform(self, X):
        newX = pd.DataFrame()
        X = clip(X)
        for feature in features:
            newX[feature] = X[feature].apply(lambda x:proba_dict_1[feature, x])
        return reshape(newX)

def normalize(df, columns):
    """
    sklearn.preprocessing.MinMaxScaler
    """
    for column in columns:
        min_val, max_val = df[column].agg([min,max])
        df[column] = (df[column] - min_val) / (max_val - min_val)
    return df

class MyTransformer4(TransformerMixin):
    def fit_transform(self, X, y=None,**fit_params):
        return self.transform(X)
    def transform(self, X):
        newX = pd.DataFrame()
        for feature in features:
            newX[feature] = X[feature].clip(lower=0).apply(lambda x:1/(x+1))
        return normalize(newX, features)
    
class MyTransformer5(TransformerMixin):
    def fit_transform(self, X, y=None,**fit_params):
        return self.transform(X)
    def transform(self, X):
        newX = pd.DataFrame()
        for feature in features:
            newX[feature] = X[feature].clip(lower=0).apply(lambda x:1/(x+1))
        return normalize(newX, features)

class MyTransformer6(TransformerMixin):
    def fit_transform(self, X, y=None,**fit_params):
        return self.transform(X)
    def transform(self, X):
        newX = pd.DataFrame()
        for feature in features:
            newX[feature] = X[feature].clip(lower=qt.loc[0.002][feature], upper=qt.loc[0.998][feature])
        return newX


In [ ]:
from sklearn.base import ClassifierMixin
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
!pip install lightautoml
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

from sklearn.pipeline import make_pipeline
my_model1 = CatBoostClassifier(
    iterations=887,
    min_child_samples=200,
    random_state=SEED,
    max_depth=6,
    verbose=1)

my_model2 = CatBoostClassifier(
    iterations=160,
    min_child_samples=200,
    max_depth=2,
    eval_metric='MultiClass',
    random_state=SEED,
    verbose=1)

initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.96,
    staircase=True)
optimizer = tf.keras.optimizers.Adam(
    learning_rate=lr_schedule, 
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name='Adam'
)
class TensorflowClassifier(ClassifierMixin):
    def __init__(self):
        self.histories=[]
        self.classes_ = [0,1,2,3,4,5,6,7,8]
        self.model = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=(75,9)),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(9, activation='softmax')
        ])
        self.model.compile(
            optimizer=optimizer, 
            loss='sparse_categorical_crossentropy',
            metrics=['sparse_categorical_crossentropy',])
    def get_params(self,deep):
        return {}
    def fit(self, X, y):
        callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)
        history = self.model.fit(X, y, epochs=300, batch_size = 75*9, validation_split=0.1, callbacks=[callback],verbose=0)
        self.histories.append(history)
        return self
    def predict_proba(self, X):
        return self.model.predict(X).reshape((-1, 9))

my_model3 = TensorflowClassifier()

my_model4 = LGBMClassifier(
    random_state=SEED,
    min_child_samples=150,
    n_estimators=74,
)

my_model5 = GradientBoostingClassifier(
    random_state=SEED,
    min_samples_leaf=200,
    n_estimators=200,
)

class AutoMLClassifier(ClassifierMixin):
    def __init__(self):
        task = Task('multiclass', metric = 'crossentropy', )
        self.model  = TabularAutoML(
            task = task,
            timeout = 300,
            general_params = {'nested_cv': False, 'use_algos': [['linear_l2', 'lgb', 'lgb_tuned']]},
            reader_params = {'cv': 5, 'random_state': SEED},
            tuning_params = {'max_tuning_iter': 20, 'max_tuning_time': 30},
            lgb_params = {'default_params': {'num_threads': 8}}, verbose=1)
        self.classes_ = [0,1,2,3,4,5,6,7,8,]
        self.train_prediction = None
    def get_params(self,deep):
        return {}
    def dataframe(self, X):
        if not isinstance(X, type(pd.DataFrame())):
            X = pd.DataFrame(X, columns=['column_{}'.format(x) for x in range(X.shape[1])])
        return X
    def fit(self, X, y):
        df = self.dataframe(X.copy())
        df['target'] = y
        self.train_prediction = self.model.fit_predict(df, roles = {'target':'target'}).data
        return self
    def predict_proba(self, X):
        X = self.dataframe(X)
        return self.model.predict(X).data

pipeline1 = make_pipeline(MyTransformer1(),my_model1)
pipeline2 = make_pipeline(MyTransformer2(),my_model2)
pipeline3 = make_pipeline(MyTransformer3(),my_model3)
pipeline4 = make_pipeline(MyTransformer4(),my_model4)
pipeline5 = make_pipeline(MyTransformer5(),my_model5)

     |████████████████████████████████| 250 kB 866 kB/s eta 0:00:01
     |████████████████████████████████| 204 kB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 424 kB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 232 kB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 100 kB 3.9 MB/s ta 0:00:011
     |████████████████████████████████| 84 kB 1.9 MB/s eta 0:00:011
     |████████████████████████████████| 90 kB 3.8 MB/s eta 0:00:011
     |████████████████████████████████| 93 kB 1.3 MB/s eta 0:00:011
     |████████████████████████████████| 94 kB 1.6 MB/s eta 0:00:011
     |████████████████████████████████| 121 kB 7.0 MB/s eta 0:00:01
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=eb02fca4353a5872b73dcfc849df568e7d2e0c1b11ccd2e6e93c703ddb6bfe4c
  Stor

In [ ]:
my_final_estimator = AutoMLClassifier()

In [ ]:
from sklearn.ensemble import VotingClassifier,StackingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss,accuracy_score

voting_estimators = [
    ('mod1', pipeline1),
    ('mod2', pipeline2),
    ('mod3', pipeline3),
    ('mod4', pipeline4),
]
stacking_estimators = [
    ('mod1', pipeline1),
    ('mod2', pipeline2),
    ('mod3', pipeline3),
    ('mod4', pipeline4),
    ('mod5', pipeline5),
]

X = train[features]
y = train['target']

mod_vot = VotingClassifier(
    estimators=voting_estimators,
    voting = 'soft',
).fit(X, y)

mod_stk = StackingClassifier(
    estimators=stacking_estimators,
    final_estimator=my_final_estimator,
    stack_method='predict_proba',
    cv=4,
).fit(X, y)

Learning rate set to 0.114731
0:	learn: 2.1023239	total: 553ms	remaining: 8m 10s
1:	learn: 2.0341029	total: 1.01s	remaining: 7m 25s
2:	learn: 1.9827364	total: 1.43s	remaining: 7m
3:	learn: 1.9430833	total: 1.83s	remaining: 6m 43s
4:	learn: 1.9117404	total: 2.26s	remaining: 6m 39s
5:	learn: 1.8872444	total: 2.65s	remaining: 6m 28s
6:	learn: 1.8656965	total: 3.1s	remaining: 6m 29s
7:	learn: 1.8478639	total: 3.57s	remaining: 6m 31s
8:	learn: 1.8335554	total: 4s	remaining: 6m 30s
9:	learn: 1.8209926	total: 4.57s	remaining: 6m 40s
10:	learn: 1.8112447	total: 5.08s	remaining: 6m 44s
11:	learn: 1.8029042	total: 5.58s	remaining: 6m 47s
12:	learn: 1.7960551	total: 6.03s	remaining: 6m 45s
13:	learn: 1.7897994	total: 6.47s	remaining: 6m 43s
14:	learn: 1.7845652	total: 6.9s	remaining: 6m 41s
15:	learn: 1.7800390	total: 7.31s	remaining: 6m 38s
16:	learn: 1.7763955	total: 7.69s	remaining: 6m 33s
17:	learn: 1.7730126	total: 8.12s	remaining: 6m 32s
18:	learn: 1.7700544	total: 8.53s	remaining: 6m 29s
1

In [ ]:
y_pred_test  = (mod_vot.predict_proba(test[features]) + mod_stk.predict_proba(test[features])) / 2
submission = test[['id']].copy()
submission['Class_1'] = y_pred_test[:,0]
submission['Class_2'] = y_pred_test[:,1]
submission['Class_3'] = y_pred_test[:,2]
submission['Class_4'] = y_pred_test[:,3]
submission['Class_5'] = y_pred_test[:,4]
submission['Class_6'] = y_pred_test[:,5]
submission['Class_7'] = y_pred_test[:,6]
submission['Class_8'] = y_pred_test[:,7]
submission['Class_9'] = y_pred_test[:,8]
submission.to_csv('submission.csv', index=False)